In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [6]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\René\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteD2OOC1\imdb_reviews-train.…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\René\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteD2OOC1\imdb_reviews-test.t…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\René\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteD2OOC1\imdb_reviews-unsupe…

Dataset imdb_reviews downloaded and prepared to C:\Users\René\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [7]:
train_data.element_spec
train_data.output_shapes
train_data.output_types

AttributeError: '_PrefetchDataset' object has no attribute 'output_shapes'